In [1]:
import boto3
import pandas as pd
from io import BytesIO

In [2]:
# variáveis de entrada
nome_bucket = 'elasticbeanstalk-us-east-2-047710767346'
subpasta = 'Case_A3'
pasta_origem = f'{subpasta}/arquivos_extraidos'
pasta_destino = f'{subpasta}/arquivos_extraidos_parquet'

# instancia do cliente
s3 = boto3.client('s3')

In [ ]:
# listando todos os objetos na pasta de origem

arquivos = s3.list_objects_v2(Bucket=nome_bucket, Prefix = f'{pasta_origem}/')

for arquivo in arquivos['Contents']:
    print(arquivo)

In [5]:
# lista para armazenar todas as pastas:

pastas = []

# lista para armazenar os arquivos target - na subpasta dados/DADOS/Dados

arquivo_dados = []

# agora, vamos analisar a resposta retornada na nossa listagem de arquivos na pasta de origme

for arquivo in arquivos.get('Contents', []):
    # verificar se o objeto é uma pasta (diretório)
    if arquivo['Key'].endswith('/'): 
        pastas.append(arquivo['Key']) # se o objet for uma pasta, armazena o nome na lista de pastas
    else:        
        if '/DADOS/' in arquivo['Key'].upper(): # se o objeto for um arquivo, verifique se está na subpasta desejada  
            if arquivo['Key'].endswith('.CSV'):
                arquivo_dados.append(arquivo['Key'])

In [6]:
arquivo_dados

['Case_A3/arquivos_extraidos/1995_microdados_censo_da_educacao_superior_1995/Microdados do Censo da Educaç╞o Superior 1995/DADOS/GRADUACAO_PRESENCIAL.CSV',
 'Case_A3/arquivos_extraidos/1995_microdados_censo_da_educacao_superior_1995/Microdados do Censo da Educaç╞o Superior 1995/DADOS/INSTITUICAO.CSV',
 'Case_A3/arquivos_extraidos/1996_microdados_censo_da_educacao_superior_1996/Microdados do Censo da Educaç╞o Superior 1996/DADOS/GRADUACAO_PRESENCIAL.CSV',
 'Case_A3/arquivos_extraidos/1996_microdados_censo_da_educacao_superior_1996/Microdados do Censo da Educaç╞o Superior 1996/DADOS/INSTITUICAO.CSV',
 'Case_A3/arquivos_extraidos/1997_microdados_censo_da_educacao_superior_1997/Microdados do Censo da Educaç╞o Superior 1997/DADOS/GRADUACAO_PRESENCIAL.CSV',
 'Case_A3/arquivos_extraidos/1997_microdados_censo_da_educacao_superior_1997/Microdados do Censo da Educaç╞o Superior 1997/DADOS/INSTITUICAO.CSV',
 'Case_A3/arquivos_extraidos/1998_microdados_censo_da_educacao_superior_1998/Microdados do 

In [ ]:
# agora, vamos iterar sobre os arquivos CSV e converter para Parquet

for arquivo_csv in arquivo_dados:
    # para obter dados do arquivo CSV, vamos fazer um get_object
    response = s3.get_object(Bucket=nome_bucket, Key=arquivo_csv)
    content = response['Body'].read()

    # agora, vamos converter os CSVs para DataFrames usando pandas
    df = pd.read_csv(BytesIO(content), encoding='latin1', sep='|')

    # requisito do negócio: converter nomes das colunas para minúsculas
    df.columns = df.columns.str.lower()

    # agora, vamos definir o nome do arquivo Parquet - tomando cuidado para pegar APENAS o nome do arquivo, não o caminho

    # primeiro, isolando o ano, para garantir que cada arquivo seja único

    ano = arquivo_csv.split('/')[-4].split('_')[-1]

    arquivo_parquet = f"{ano}_{arquivo_csv.split('/')[-1].replace('.CSV', '.parquet')}"

    # escrevendo dados Parquet diretamente no S3, sem armazenar o íncice
    buffer = BytesIO()
    df.to_parquet(buffer, index=False)

    # Enviar dados Parquet para o S3
    s3.put_object(Body=buffer.getvalue(), Bucket=nome_bucket, Key=f'{pasta_destino}/{arquivo_parquet}')
    print(f'Arquivo convertido com sucesso: {pasta_destino}/{arquivo_parquet}')

In [22]:
arquivo_csv

'Case_A3/arquivos_extraidos/1997_microdados_censo_da_educacao_superior_1997/Microdados do Censo da Educaç╞o Superior 1997/DADOS/GRADUACAO_PRESENCIAL.CSV'

In [24]:
ano = arquivo_csv.split('/')[-4].split('_')[-1]

'1997'

In [7]:
# listando arquivos na pasta destino

arquivos_parquet = s3.list_objects_v2(Bucket=nome_bucket, Prefix = f'{pasta_destino}/')

In [11]:
lista_parquet = []

for aquivo in arquivos_parquet:
    arquivo = arquivos_parquet['Contents']
    lista_parquet.append(arquivo)


In [9]:
lista_parquet

[[{'Key': 'Case_A3/arquivos_extraidos_parquet/',
   'LastModified': datetime.datetime(2023, 12, 11, 11, 55, 25, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Size': 0,
   'StorageClass': 'STANDARD'},
  {'Key': 'Case_A3/arquivos_extraidos_parquet/1995_GRADUACAO_PRESENCIAL.parquet',
   'LastModified': datetime.datetime(2024, 1, 21, 14, 4, 18, tzinfo=tzutc()),
   'ETag': '"fd021c263362bd12a6a38b673bab405f"',
   'Size': 360495,
   'StorageClass': 'STANDARD'},
  {'Key': 'Case_A3/arquivos_extraidos_parquet/1995_INSTITUICAO.parquet',
   'LastModified': datetime.datetime(2024, 1, 21, 14, 4, 21, tzinfo=tzutc()),
   'ETag': '"05a8650e7d3bb4ee2d8f38788a382dda"',
   'Size': 72468,
   'StorageClass': 'STANDARD'},
  {'Key': 'Case_A3/arquivos_extraidos_parquet/1996_GRADUACAO_PRESENCIAL.parquet',
   'LastModified': datetime.datetime(2024, 1, 21, 14, 4, 28, tzinfo=tzutc()),
   'ETag': '"e05e61ac438cecf75c26061964ae4195"',
   'Size': 140808,
   'StorageClass': 'STANDARD'},
  {'Ke

In [14]:
# agora, vamos testar e ler um arquivo parquet 

import pyarrow.parquet as pq
import s3fs

# Substitua 'seu_arquivo.parquet' pelo nome do seu arquivo Parquet

caminho_s3 = 'Case_A3/arquivos_extraidos_parquet/1995_GRADUACAO_PRESENCIAL.parquet'



In [15]:
# Criando o objeto S3FileSystem
s3 = s3fs.S3FileSystem()

# Lendo o arquivo Parquet diretamente do S3
tabela_parquet = pq.read_table(f's3://{nome_bucket}/{caminho_s3}', filesystem=s3)

# Convertendo a tabela para um DataFrame do pandas
df_parquet = tabela_parquet.to_pandas()


In [16]:
df_parquet

,mascara,nr_ano,cd_curso_seec,cd_curso,codmunic,eh_capital,no_regiao,sg_uf_curso,no_uf,no_curso,...,qt_ingr_outros_femi_diurno,qt_ingr_outros_masc_diurno,qt_vagas_noturno,qt_insc_1opc_noturno,qt_insc_2opc_noturno,qt_aprovados_noturno,qt_ingr_vest_femi_noturno,qt_ingr_vest_masc_noturno,qt_ingr_outros_femi_noturno,qt_ingr_outros_masc_noturno
0,1,1995,301,2020000,NaN,N,CENTRO-OESTE,GO,Goias,CIENCIA DA COMPUTACAO,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,302,2066000,NaN,N,SUDESTE,MG,Minas Gerais,LETRAS,...,0,0,60,58,0,58,42,4,1,0
2,2,1995,303,2050000,NaN,N,SUDESTE,MG,Minas Gerais,ESTUDOS SOCIAIS,...,0,0,60,41,0,41,25,16,0,0
3,2,1995,301,2018000,NaN,N,SUDESTE,MG,Minas Gerais,CIENCIAS,...,0,0,60,50,0,50,36,4,1,0
4,3,1995,302,2021000,NaN,N,SUDESTE,RJ,Rio de Janeiro,CIENCIAS CONTABEIS,...,0,0,50,160,139,50,29,21,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7084,893,1995,341,2059000,NaN,S,SUDESTE,SP,Sao Paulo,GEOLOGIA - SAO PAULO,...,0,0,0,0,0,0,0,0,0,0
7085,893,1995,357,2090000,NaN,N,SUDESTE,SP,Sao Paulo,QUIMICA - RIBEIRAO PRETO,...,0,0,0,0,0,0,0,0,0,0
7086,893,1995,383,2001010,NaN,S,SUDESTE,SP,Sao Paulo,ADMINISTRACAO DE EMPRESAS - RIB PRETO,...,0,0,40,579,0,128,14,26,0,0
7087,893,1995,350,2022000,NaN,S,SUDESTE,SP,Sao Paulo,CIENCIAS ECONOMICAS - SAO PAULO,...,0,0,90,962,924,0,14,76,0,0
